In [1]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('../01_preprocess/preprocessed.csv')

df.drop(['cycle', 'instr', 'LLC_miss','CPU','L1_miss', 'br_miss', 'scale', 'counted_final_key_count', 'GHz', 'task', 'IPC'], axis=1, inplace=True)
df.drop(['nodeCount_Leaf', 'nodeCount_Inner', 'nodeCount_Dense', 'nodeCount_Hash', 'nodeCount_Head4', 'nodeCount_Head8', 'nodeCount_Dense2'], axis=1, inplace=True)

df.head()

,config_name,const_basicHintCount,const_enableBasicHead,const_enableDense,const_enableDense2,const_enableDensifySplit,const_enableHash,const_enableHashAdapt,const_enableHeadNode,const_enablePrefix,...,data_name,data_size,data_sorted,density,op,payload_size,rand_seed,ycsb_range_len,ycsb_zipf,time
0,hash,16,1,0,0,0,1,0,0,1,...,data/urls,5798920.0,0.0,0.531252,ycsb_c_init,8.0,1.701944e+09,100.0,1.133614,3.923144e-07
1,hash,16,1,0,0,0,1,0,0,1,...,data/urls,5798920.0,0.0,0.531252,ycsb_c,8.0,1.701944e+09,100.0,1.133614,1.944000e-07
2,dense3,16,1,1,0,1,0,0,0,1,...,data/urls,3175985.0,0.0,0.641406,ycsb_c_init,8.0,1.701944e+09,100.0,0.316120,4.814254e-07
3,dense3,16,1,1,0,1,0,0,0,1,...,data/urls,3175985.0,0.0,0.641406,ycsb_c,8.0,1.701944e+09,100.0,0.316120,5.009000e-07
4,hints,16,1,0,0,0,0,0,0,1,...,data/urls,4933290.0,0.0,0.876636,ycsb_e_init,8.0,1.701944e+09,100.0,0.666485,5.052004e-07


In [2]:
# Identify non-numeric columns
non_numeric_columns = df.select_dtypes(['object']).columns

print("Number of columns:", df.shape[1])

# Apply one-hot encoding to non-numeric columns
df = pd.get_dummies(df, columns=non_numeric_columns)

print("Number of columns after one-hot:", df.shape[1])

Number of columns: 28
Number of columns after one-hot: 33


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('time', axis=1), df['time'], test_size=0.2, random_state=42)

# Create a random forest regressor
reg = RandomForestRegressor()

# Fit the regressor to the training data
reg.fit(X_train, y_train)

# Predict the percentile brackets for the test data
y_pred = reg.predict(X_test)

# Evaluate the performance of the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("MSE:",mse)
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)



MSE: 1.7388941088001857e-16
R-squared: 0.9975435424783912


In [5]:
print("Done")

Done
